## get embedding model

In [1]:
from pyapp.model_connection.model import  get_model_embeddings_from_config_dir
from pathlib import Path
here = Path("./").resolve()
config_dir = here / "../"
model = get_model_embeddings_from_config_dir(config_dir=config_dir)




May 14, 2025 > 11:40:26 |  pyapp.utils.config : 10 |   ℹ️ INFO  |  Searching for config file appdeps.toml  
May 14, 2025 > 11:40:26 |  pyapp.utils.config : 12 |   🐞 DEBUG  |  Checking /Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/projects/py/projects/airplane_simple_retriever/notebooks/../appdeps.toml  


## ingest and store

In [6]:
raw_data_path.resolve()

PosixPath('/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/projects/py/projects/airplane_simple_retriever/data/raw_data')

In [8]:
from pyapp.vectordb import get_vector_store
vector_store_path = here / "../data/vectordb/test.db"
raw_data_path = here / "../data/raw_data/test.csv"
table_name = "test"
vector_store , conn = get_vector_store(database_path= str(vector_store_path) , table_name=table_name , embedding_model=model)

from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path=str(raw_data_path.resolve()))
data = loader.load()
data


[Document(metadata={'source': '/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/projects/py/projects/airplane_simple_retriever/data/raw_data/test.csv', 'row': 0}, page_content='data: i feel good\n: '),
 Document(metadata={'source': '/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/projects/py/projects/airplane_simple_retriever/data/raw_data/test.csv', 'row': 1}, page_content='data: i feel bad\n: '),
 Document(metadata={'source': '/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/projects/py/projects/airplane_simple_retriever/data/raw_data/test.csv', 'row': 2}, page_content='data: the capital of france is paris\n: '),
 Document(metadata={'source': '/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/projects/py/projects/airplane_simple_retriever/data/raw_data/test.csv', 'row': 3}, page_content='data: good food\n: ')]

In [9]:
vector_store.add_documents(data)


['8b8ab1b5-b9e6-413c-8d96-ff8585b33e08',
 '1ab4ccfb-48f9-44bc-a92e-c842b6c76e0d',
 '8ba1c660-db10-484e-accb-9abf41e89402',
 '7a7717e3-94dc-4e14-9c38-d3a0be64a605']

In [12]:
query = "what is the capital of france?"
vector_store.similarity_search(query , k=1)[0].page_content






'data: the capital of france is paris\n: '

## with observation

In [22]:
from pyapp.observation.phoneix import traced_agent

retriever = vector_store.as_retriever(search_kwargs={"k": 1})

@traced_agent(name="airplane-simple-retriever")
def agent(messages: list[dict], session_id: str) -> str:
    question = messages[-1]["content"]
    return retriever.invoke(question)


In [23]:
import uuid
session_id = str(uuid.uuid4())

messages = [
  {"role": "user", "content": "what is the capital of france?"}
]
answer, link = agent(messages, session_id)

print(answer)
print(link)

Invalid type Document in attribute 'output.value' value sequence. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or None


[Document(metadata={'source': '/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/projects/py/projects/airplane_simple_retriever/data/raw_data/test.csv', 'row': 2, '_similarity_score': 0.9098639438966993}, page_content='data: the capital of france is paris\n: ')]
http://localhost:6006/projects/UHJvamVjdDoy/traces/84c839e0eaa034dd23039b7d2b5fb5ae?selectedSpanNodeId=U3Bhbjoy


In [12]:
from pyapp.observation.phoenix import observation


ModuleNotFoundError: No module named 'pyapp.observation.phoenix'